In [ ]:
##### IMPORTS #####
import pandas as pd
import numpy as np
import re
from io import BytesIO
from google.cloud import storage
client = storage.Client()
import matplotlib.pyplot as plt
from datetime import datetime
from MAI2023.masterFunctions_v20240502 import *
import geopandas as gpdcheckRunningOrders

In [ ]:
def infoVars(df, mktID, locGroup): # assign info variables based on date and location
    df['mktID'] = mktID
    df['locGroup'] = locGroup
    country = checkLocationFileStatus(mktID, 'country')
    df['country'] = country
    display(df['ident'])
    try: # Necessary because some exports have band names starting with 1_ or 2_, not the date. Comes from merge of two image collections ic_old and ic_new
        df['date'] = pd.to_datetime(df['ident'].apply(lambda x: datetime.strptime(x[:8], "%Y%m%d").date()))
    except:
        df['date'] = pd.to_datetime(df['ident'].apply(lambda x: datetime.strptime(x[2:10], "%Y%m%d").date()))
    try:
        df['time'] = df['ident'].apply(lambda x: datetime.strptime(x[9:15], "%H%M%S").time())
    except:
        df['time'] = df['ident'].apply(lambda x: datetime.strptime(x[11:17], "%H%M%S").time())
        
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    #df['time_decimal']=   df['time'].dt.hour + df['time'].dt.minute / 60 + df['time'].dt.second / 3600
    df['time_decimal'] = df['time'].apply(lambda t: t.hour + t.minute / 60 + t.second / 3600)
    df['weekday'] = (df['date'].dt.weekday + 1) % 7
    df['mkt_lat'] = pd.to_numeric(df['mktID'].str.extract(r'lon(-?\d+)_(\d+)').apply(lambda x: f"{x[0]}.{x[1]}", axis=1))
    df['mkt_lon'] = pd.to_numeric(df['mktID'].str.extract(r'lat(-?\d+)_(\d+)').apply(lambda x: f"{x[0]}.{x[1]}", axis=1))
    if country=="Kenya": # For some locations in Kenya, the lon and lat coordinates were flipped in their mktid
        df['origLat'] = df['mkt_lat']
        df.loc[df['mkt_lat'] > 30, 'mkt_lat'] = df['mkt_lon']
        df.loc[df['mkt_lon'] < 30, 'mkt_lon'] = df['origLat']
        df.drop(columns=['origLat'], inplace=True)
    if country=="Ethiopia": # For some locations in Ethiopia, the lon and lat coordinates were flipped in their mktid
        df['origLat'] = df['mkt_lat']
        df.loc[df['mkt_lat'] > 20, 'mkt_lat'] = df['mkt_lon']
        df.loc[df['mkt_lon'] < 20, 'mkt_lon'] = df['origLat']
        df.drop(columns=['origLat'], inplace=True)
    return df

def identifyMktDays(df,minRank, threshold_for_market): # identify market days based on detected areas and their threshold values
    print('minRank: ', minRank, ', Threshold_for_market: ', threshold_for_market)
    # List all maximum threshold values on the days-of-week where we detected something and that detection falls below a threshold 
    min_thres_by_day = df[df['strictnessRank'] <= threshold_for_market].groupby('weekdayThisAreaIsActive')['strictnessRank'].min()
    if loc in invalid_market_days: # remove  known misdetections from visual checks of outputs
        value_to_remove = invalid_market_days[loc]
        if value_to_remove in min_thres_by_day:
            del min_thres_by_day[value_to_remove]
    print('strictness rank and active weekdays',min_thres_by_day)
    # Find the clearest detection 
    lowest_thres = min_thres_by_day.min()
    print('lowest strictness rank',lowest_thres)
    # Filter unique days of week where the threshold is within 3 ranks of the lowest threshold value -> identifies all similarly high detections
    localMktDays = list(min_thres_by_day[min_thres_by_day - lowest_thres <= 3].index.unique())
    print('localMktDays', localMktDays)
    def find_position(weekday):
        try:
            return list(localMktDays).index(weekday)
        except ValueError:
            return -1  # Return 0 if the weekday is not found in the list
    df['pos'] = df['weekday'].apply(find_position)
    df['mktDay'] = None
    df.loc[(df['weekday'] == df['weekdayThisAreaIsActive']) & (df['pos'] >= 0), 'mktDay'] = 1 # detected market day
    df.loc[(df['weekday'] != df['weekdayThisAreaIsActive']) & (df['pos'] == -1), 'mktDay'] = 0 # detected non-market day
    df.loc[(df['weekday'] != df['weekdayThisAreaIsActive']) & (df['pos'] >= 0), 'mktDay'] = 99 # observation of detected market area for a given weekday on a different weekday
    return df

def cleanActMeasures(df, geos, varsOfInterest): 
    # Set values to NA that exceed the median value per market, weekday of operation
    # and instrument by more than twice the IQR , calculated over the period 
    # outside Covid and for typical times and good images
    #df['time_decimal'] = df['ident'].apply(extract_time_decimal)
    df['diff_to_median_time'] = df.apply(lambda row: abs(row['time_decimal'] - df['time_decimal'].median()), axis=1)
    mask = (
        (df['date'].between('2020-03-01', '2021-02-28')) | # potentially covid-affected
        (df['date'] < '2018-01-01') |                      # generally noisier because of sparse imagery
        (df['diff_to_median_time'] > .5) |                  # differing sun angle
        ((df['clear_percent'].notnull()) & (df['clear_percent'] < 10)) | # noisy imagery
        ((df['cloud_percent'].notnull()) & (df['cloud_percent'] > 50))
    )
    # Create a new column 'exclDates' based on the mask
    df['exclDates'] = mask.astype(int)
    for b in geos: # within each possible area
        df[f'sumsum_maxpMax_{b}'] = df[f'sumsum_maxpMax_{b}'] / df[f'ccount_maxpMax_{b}'] # convert sum variable into mean deviations

        # Typical number of pixels per shape
        max_count = df.loc[df['exclDates'] != 1].groupby(['weekdayThisAreaIsActive', 'mktDay'])[f'ccount_maxpMax_{b}'].max().reset_index()
        df = pd.merge(df, max_count, on=[ 'weekdayThisAreaIsActive', 'mktDay'], how='outer', suffixes=('', '_max_count'))        

        for p in varsOfInterest:
            try:
                # set to NA those values coming from images that cover less than 50% of the typical footprint
                df.loc[df[f'ccount_maxpMax_{b}']  < 0.5 *(df[f'ccount_maxpMax_{b}_max_count']), f'{p}_maxpMax_{b}'] = np.nan

                # calculate median, iqr by detected area and sensor, and merge to dataframe
                median = df.loc[df['exclDates'] != 1].groupby(['weekdayThisAreaIsActive', 'mktDay', 'instrument'])[f'{p}_maxpMax_{b}'].quantile(0.5).reset_index()
                df = pd.merge(df, median, on=[ 'weekdayThisAreaIsActive', 'mktDay', 'instrument'], how='outer', suffixes=('', '_median'))

                p25 = df.loc[df['exclDates'] != 1].groupby(['weekdayThisAreaIsActive', 'mktDay', 'instrument'])[f'{p}_maxpMax_{b}'].quantile(0.25)
                p75 = df.loc[df['exclDates'] != 1].groupby(['weekdayThisAreaIsActive', 'mktDay', 'instrument'])[f'{p}_maxpMax_{b}'].quantile(0.75)
                iqr = (p75-p25).reset_index()
                df = pd.merge(df, iqr, on=[ 'weekdayThisAreaIsActive', 'mktDay', 'instrument'], how='outer', suffixes=('', '_iqr'))
                
                # set to NA those values that are more than twice the IQR above the median
                df.loc[df[f'{p}_maxpMax_{b}']  > (df[f'{p}_maxpMax_{b}_median'] + 2 * df[f'{p}_maxpMax_{b}_iqr']), f'{p}_maxpMax_{b}'] = np.nan
                df = df.drop([f'{p}_maxpMax_{b}_median', f'{p}_maxpMax_{b}_iqr'], axis=1)    

            except Exception as e:
                print('Error in cleanActMeasures', e)
                pass
    return df

def contains_substring(s, substrings):
    for substring in substrings:
        if substring in s:
            return True
    return False

def drop_columns_by_pattern(df, patterns_to_drop):
    for pattern in patterns_to_drop:
        try:
            df = df.drop(df.filter(like=pattern).columns, axis=1)
        except Exception as e:
            print(f"Error occurred while dropping columns for pattern '{pattern}': {e}")
    return df

def determine_sensor(row):
    image_id = row['ident']
    condition1 = '3B' in image_id[-2:]
    condition2 = '_1_' in image_id
    if condition1 or condition2:
        return 'PS2'
    else:
        return 'PSB.SD'
    
def prepare_properties(locGroup, loc, propToDrop):
    df_prop = pd.read_csv(f'gs://exports-mai2023/{locGroup}/properties/propEx_{locGroup}_{loc}.csv')
    
    # Extract 'ident' from 'system:index' column
    df_prop['ident'] = df_prop['system:index'].str.slice(stop=23) 
    # Determine the imagery generation of each image
    df_prop['instrument'] = df_prop.apply(determine_sensor, axis=1)
    print(df_prop['instrument'].unique())
    # Drop specified properties from the DataFrame
    for prop in propToDrop:
        try:
            df_prop = df_prop.drop(prop, axis=1)
        except KeyError:
            pass
    return df_prop  

def identify_varying_areas(wide_df): # Identify the largest ring in which P75 non-market day activity still does not exceed P50 market day activity
    market_days = wide_df.loc[wide_df['mktDay'] == 1, 'weekday'].unique().tolist()
    gdfs = [] # dataframe to hold the selected shapes
    for market_day in market_days:
        print('market_days', market_days, market_day)

        df_mktDays = wide_df[(wide_df['mktDay'] == 1) 
                     & (wide_df['exclDates'] == 0) 
                     & (wide_df['clear_percent'] > 90) 
                     & (wide_df['weekdayThisAreaIsActive']==market_day) 
                     & (wide_df['weekday']==market_day) 
                     & (wide_df['diff_to_median_time'] <.5)]
        
        filtered_columns_sum = df_mktDays.loc[:, df_mktDays.columns.str.contains('sumsum') & 
                                     ~df_mktDays.columns.str.contains('_100')]

        # Exclude columns that are all NA
        filtered_columns_sum = filtered_columns_sum.loc[:, filtered_columns_sum.notna().any()].columns.tolist()
        
        df_nonmktDays = wide_df[(wide_df['mktDay'] == 0) 
                                & (wide_df['exclDates'] == 0) 
                                & (wide_df['clear_percent'] > 90)
                                & (wide_df['diff_to_median_time'] <.5) 
                                & (wide_df['weekdayThisAreaIsActive']==market_day)]
        p75_nonmktDays_sum = df_nonmktDays[filtered_columns_sum].dropna(subset=filtered_columns_sum, how='all').quantile(0.75)    
    
        # keep high quality images, separately for market and non-market days

        # Calculate variance and mean for percentiles (filtered_columns_p)
        p50_mktDays_sum = df_mktDays[filtered_columns_sum].dropna(subset=filtered_columns_sum, how='all').quantile(0.5)
        result = pd.concat([p50_mktDays_sum, p75_nonmktDays_sum], axis=1)
        result.columns = ['p50_mktDays_sum', 'p75_nonmktDays_sum']

        print(result)
        first_row_index = (result['p75_nonmktDays_sum'] > result['p50_mktDays_sum']).replace(False, np.nan).idxmax()
        if pd.isna(first_row_index):
            first_row_index= result.iloc[-1].name

        print("First row where p75_nonmktDays_sum > p50_mktDays_sum: ",first_row_index)

        # Update DataFrame with name of area per weekday that we consider the market area
        wide_df[f'maxVar_s_{market_day}_maxpMax'] = first_row_index
        #print(loc,first_row_index)
        filtered_gdf = select_areas(market_day, first_row_index)
        gdfs.append(filtered_gdf)

    return wide_df, gdfs

def select_areas(market_day,first_row_index): #select the shapes associated with the selected market area
    # extract substring between second last and last instance of _
    temp = first_row_index.split('_')
    if len(temp) >= 2:
        minRing =  int(temp[-2])
    else:
        minRing = None  # Return None if there aren't enough parts
    print('minRing', minRing)
    # load shapefile    
    shp_path = f'gs://exports-mai2023/{locGroup}/shapes/shp_MpM6_{locGroup}{loc}.shp'
    gdf = gpd.read_file(shp_path)    
    filtered_gdf = gdf[(gdf['weekdayShp'] == market_day) & 
                   (gdf['strictness'] == minRing) & 
                   (gdf['subStrictn'] == 100)].copy()
    #filtered_gdf.plot()
    filtered_gdf.loc[:, 'mktid'] = loc  # Use .loc to set values
    return filtered_gdf

def prepend_zero_if_single_digit(value):
    if len(str(value)) == 1:
        return '0' + str(value)
    else:
        return str(value)


In [ ]:
## Identify locations
#cnx = mysql.connector.connect(user='root',password='XXX',host='XXX',database='mai-database')

query = '''
SELECT l.Location
FROM `mai-database`.`location_file` l
WHERE EXISTS (
    SELECT 1
    FROM `mai-database`.`process_runs` pr
    WHERE pr.Location = l.Location
    AND pr.Process = '04ActivityExport'
    AND pr.Setup = 'exportAct5'
    AND pr.Status = 'complete'
)
'''
cursor = cnx.cursor()
cursor.execute(query)
response = cursor.fetchall()
cursor.close()
cnx.close()

locs = [row[0] for row in response]
print(locs, len(locs))


In [ ]:
print(locs, len(locs))
locs=list(set(locs))
freqDayStr_short='w7'
maxRank = 4 # exclude altitude levels above this
ring_area_share = 0.8  # Only consider rings whose area is more than 100(1-X)% of the shape defining the outer border of the ring

prefix = "exports-mai2023"
target_folder ='activity_cleaned_2024'
varsOfInterest=['p50','sumsum', 'ccount']
threshold_for_market=24
bucket=client.get_bucket('exports-mai2023')

forMerge=['ident','weekdayThisAreaIsActive','date','mktDay','mktID','locGroup','time','year','month', 'weekday', 'mkt_lat','mkt_lon','time_decimal'] 
patterns_to_drop = ['ground_control','strictnessRank', 'subStrictnessRank''Geography','origName_', 'coorLength_', '.geo', 'system:index_b0', 'system:index', 'weekday_','market']
propToDrop=['quality_category','system:index', '.geo','order_id', 'pixel_resolution','gsd','provider', 'published', 'publishing_stage', 'item_type', 'item_id', 'snow_ice_percent', 'strip_id','updated']
print(locs, len(locs))



In [ ]:
# Initialize an empty list to store DataFrames from each location
list_df_acrossLocs = []
pd.set_option('display.width', 100)  # Set display width
pd.set_option('display.max_columns', 500)  # Show all columns
    
def check_file_exists(bucket, file_path):
    """Check if a file exists in a Google Cloud Storage bucket."""
    blob = bucket.blob(file_path)
    return blob.exists()

invalid_market_days = {
    'lon37_6468lat0_0505': 0, # small parking lot outside football stadion
    'lon39_1245lat-4_5529': 3 # noise 
}

problemLocs=[]

locCount=0
for loc in sorted(locs, reverse=False):
    print('loc', loc)
    locCount=locCount+1
    if check_file_exists(client.get_bucket('exports-mai2023'), f"{target_folder}/df_{loc}.csv") and check_file_exists(client.get_bucket('exports-mai2023'), f"{target_folder}/shp_{loc}.shp"):
        #print(f"The files df_{loc}.csv and df_{loc}.shp already exist.")
        pass
        
    else:    
        try:
            # Get the GEEbucket and locGroup for the current location
            GEEbucket = checkLocationFileStatus(loc, 'bucket')
            locGroup = checkLocationFileStatus(loc, 'locGroup')
            print(loc, GEEbucket, locGroup)
            # prepare image property dataframe to be merged in later
            df_prop = prepare_properties(locGroup, loc, propToDrop)

            # Read the activity CSV file
            df = pd.read_csv(f'gs://exports-mai2023/{locGroup}/measures/exportAct5_maxpMax{loc}_{freqDayStr_short}.csv')
            
            # keep only entries that fall between the strictest rank we define and the least strict one for a given shape, but at least 30
            minRank = max(df['strictnessRank'].min(),30)
            df = df[(df['strictnessRank'] <= minRank) & (df['strictnessRank'] >= maxRank)]
            df = df[((df['subStrictnessRank'] <= minRank) & (df['subStrictnessRank'] > maxRank)) | (pd.isna(df['subStrictnessRank'])) | (df['subStrictnessRank'] ==100) ]
            print(df['strictnessRank'].unique().tolist())
            df['subStrictnessRank'] = df['subStrictnessRank'].fillna(100).astype(int)

            eligible_rings = df[df['subStrictnessRank'] != 100].groupby('strictnessRank', as_index=False)['subStrictnessRank'].max()
            additional_rows = pd.DataFrame({
                'strictnessRank': df['strictnessRank'].unique(),
                'subStrictnessRank': 100
            })
            eligible_shapes = pd.concat([eligible_rings, additional_rows]).sort_values(by='strictnessRank').reset_index(drop=True)

            df_elig =  pd.merge(df, eligible_shapes, on=['strictnessRank', 'subStrictnessRank'])

            df_elig.rename(columns={'weekdayShp': 'weekdayThisAreaIsActive'}, inplace=True)

            # Extract image id 
            df_elig['ident'] = df_elig['ident'].str.rsplit('_maxpMax', n=1).str[0].str[1:] 
            df_elig['weekdayThisAreaIsActive'] = df_elig['weekdayThisAreaIsActive'].astype(int)
            df_elig['strictnessRank'] = df_elig['strictnessRank'].astype(int)

            # Create area_id column from the strictnessRank variables
            df_elig['strictnessRank_str'] = df_elig['strictnessRank'].apply(prepend_zero_if_single_digit)
            df_elig['subStrictnessRank_str'] = df_elig['subStrictnessRank'].apply(prepend_zero_if_single_digit)
            df_elig['area_id'] = df_elig['strictnessRank_str'].astype(str) + '_' + df_elig['subStrictnessRank_str'].astype(str)

            geos = df_elig['area_id'].unique()

            # Append area id to variable names
            new_column_names = {old_col: old_col + '_maxpMax'  for old_col in varsOfInterest}
            df_elig = df_elig.rename(columns=new_column_names)
            
            # Assign info variables
            df_elig = infoVars(df_elig, loc, locGroup)
            
            # Identify market days
            df_elig = identifyMktDays(df_elig,minRank, threshold_for_market)
            
            wide_df = df_elig.pivot_table(index=forMerge, columns='area_id', values=list(new_column_names.values()))
            wide_df.columns = ['_'.join(str(s).strip() for s in col if s) for col in wide_df.columns]
            wide_df.reset_index(inplace=True)    

            # Drop unnecessary columns
            wide_df = drop_columns_by_pattern(wide_df, patterns_to_drop)

            # Merge with properties
            wide_df = pd.merge(wide_df, df_prop, on='ident', how='left')
            
            # Exclude outliers
            wide_df = cleanActMeasures(wide_df, geos, varsOfInterest)
            
            # Identify varying areas on market days
            wide_df, market_shapes_list = identify_varying_areas(wide_df)
            
            # Upload activity CSV to GCS
            blob = bucket.blob(f"{target_folder}/df_{loc}.csv")
            blob.upload_from_string(wide_df.to_csv(index=False), content_type='text/csv')

            # Export GeoDataFrame to ESRI Shapefile format
            tmp_file = '/temp_shapefile.shp' 
            market_shapes = gpd.GeoDataFrame(pd.concat(market_shapes_list, ignore_index=True),
                                             crs=market_shapes_list[0].crs)

            market_shapes.to_file(tmp_file, driver='ESRI Shapefile')

            # Upload each file of the shapefile to GCS
            for ext in ['shp', 'shx', 'dbf', 'prj']:
                blob = bucket.blob(f"{target_folder}/shp_{loc}.{ext}")
                blob.upload_from_filename(f"{tmp_file.replace('.shp', '.' + ext)}")

            print(f'{loc} exported! {locCount}/{len(locs)}')
        except Exception as e:
            print(f'problem with {loc}',e)
            problemLocs.extend([loc])   
            pass
print(problemLocs) 


In [ ]:
## EXPORTING SHAPES
names = ["MOZ_20240702","MWI_20240702","ETH_20240702", "KEN_20240702"]
countries = ["Mozambique", "Malawi", "Ethiopia", "Kenya"]
for name, country in zip(names, countries):
    
    ## Merge all csvs from an area of interest into one master file
    dataset_name = name
    target_folder ='activity_cleaned_20240704'
    geo_of_interest = country
    level = "country"

    query = f'''
    SELECT l.Location
    FROM `mai-database`.`location_file` l
    WHERE l.{level} = '{geo_of_interest}' 
    AND EXISTS (
        SELECT 1
        FROM `mai-database`.`process_runs` pr
        WHERE pr.Location = l.Location
        AND pr.Process = '04ActivityExport'
        AND pr.Setup = 'exportAct5'
        AND pr.Status = 'complete'
    )
    '''

    cursor = cnx.cursor()
    cursor.execute(query)
    response = cursor.fetchall()
    cursor.close()
    cnx.close()

    locs = [row[0] for row in response]
    print(locs, len(locs))

    list_df_acrossLocs=[]
    loc_counter=0
    batch_counter = 0

    gdfs = []
    for loc in locs:
        print(loc)
        try:
            gdf = gpd.read_file(f"gs://exports-mai2023/{target_folder}/shp_{loc}.shp", driver='ESRI Shapefile')
            gdfs.append(gdf)   
        except Exception as e:
            print(f"Error with {loc}: {e}")
            pass
            
    merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)
    # Export the merged GeoDataFrame to a new shapefile
    merged_gdf.to_file(f'shp_{dataset_name}.shp', driver='ESRI Shapefile')


In [ ]:
## EXPORTING ACTIVITY MEASURES
## Identify locations

names = ["UG"]#, "ETH_20240702", "MOZ_20240702"]
countries = ["Uganda"]#, "Ethiopia","Mozambique"]
for name, country in zip(names, countries):
    #cnx = mysql.connector.connect(user='root',password='XXX',host='XXX',database='mai-database')

    ## Merge all csvs from an area of interest into one master file
    dataset_name = name
    target_folder ='activity_cleaned_2024'
    geo_of_interest = country
    level = "country"

    query = f'''
    SELECT l.Location
    FROM `mai-database`.`location_file` l
    WHERE l.{level} = '{geo_of_interest}' 
    AND EXISTS (
        SELECT 1
        FROM `mai-database`.`process_runs` pr
        WHERE pr.Location = l.Location
        AND pr.Process = '04ActivityExport'
        AND pr.Setup = 'exportAct5'
        AND pr.Status = 'complete'
    )
    '''

    cursor = cnx.cursor()
    cursor.execute(query)
    response = cursor.fetchall()
    cursor.close()
    cnx.close()

    locs = [row[0] for row in response]
    print(locs, len(locs))

    list_df_acrossLocs=[]
    loc_counter=0
    batch_counter = 0

    def replace_after_underscore(s):
            return s[:s.rfind('_') + 1] + '100'

    for loc in locs:
        print(loc)
        filePath = f'gs://exports-mai2023/{target_folder}/df_{loc}.csv'

        # Read the CSV file
        #try:
        df = pd.read_csv(filePath)
        df = df.drop(columns=df.filter(like='count').columns)        
        #display(df)
        loc_counter += 1
        #print(len(list(df.columns)))
        market_days = df.loc[df['mktDay'] == 1, 'weekday'].unique().tolist()
        tokeep=[]
        for market_day in market_days:
            #print(market_day)
            target_var = df[f"maxVar_s_{market_day}_maxpMax"].unique().tolist()[0].replace("maxpmax", "maxpMax")
            tokeep.extend([df[f"maxVar_s_{market_day}_maxpMax"].unique().tolist()[0].replace("maxpmax", "maxpMax")])
            df.loc[(market_day == df['weekdayThisAreaIsActive']) , 'activity_measure'] = df[target_var]

        mean_nonmktday =  df.loc[(df['instrument'] != 'PS2') & (df['mktDay']==0)].groupby(['weekdayThisAreaIsActive', 'instrument'])['activity_measure'].mean().reset_index()
        df = pd.merge(df, mean_nonmktday, on=[ 'weekdayThisAreaIsActive', 'instrument'], how='outer', suffixes=('', '_mean_nonmktday'))
        df['activity_measure_mean0'] = df['activity_measure']-df['activity_measure_mean_nonmktday']
        mean_mktday = df.loc[(df['instrument'] != 'PS2') & (df['mktDay']==1) & df['year']==2021].groupby(['weekdayThisAreaIsActive', 'instrument'])['activity_measure_mean0'].mean().reset_index()
        df = pd.merge(df, mean_mktday, on=[ 'weekdayThisAreaIsActive', 'instrument'], how='outer', suffixes=('', '_mean_mktday'))
        df['activity_measure_norm']=100*df['activity_measure_mean0']/df['activity_measure_mean0_mean_mktday']

        # Apply the function to each string in listA
        tokeep_100 = [replace_after_underscore(col) for col in tokeep]        
        cols_to_drop = [col for col in df.columns if ('maxVar' not in col and 'maxpMax' in col) and col not in tokeep and col not in tokeep_100 ]
        #print(cols_to_drop)
        df = df.drop(columns=cols_to_drop)
        df = df.drop(columns=['ground_control', 'time', 'ident', 'locGroup'])

        list_df_acrossLocs.append(df)

        #print(len(list(df.columns)))
        print('here', loc_counter)
        #stop
        # Export and reset for every 50 locations
        if loc_counter % 50 == 0:
            df_acrossLocs = pd.concat(list_df_acrossLocs, ignore_index=True)

            # Rearrange column order
            new_order = ['mktID', 'weekdayThisAreaIsActive']
            df_acrossLocs = df_acrossLocs[new_order + [col for col in df_acrossLocs.columns if col not in new_order]]

            # Save to CSV
            df_acrossLocs.to_csv(f'df_{dataset_name}_batch{batch_counter}.csv', index=False)
            print('saved batch', batch_counter)
            # Reset lists and counters for the next batch
            list_df_acrossLocs = []
            batch_counter += 1

        #except Exception as e:
        #    print(f"Error with {loc}: {e}")
        #    pass

    # Handle the final batch if it has fewer than 50 locations
    if list_df_acrossLocs:
        df_acrossLocs = pd.concat(list_df_acrossLocs, ignore_index=True)

        # Rearrange column order
        new_order = ['mktID',  'weekdayThisAreaIsActive']
        df_acrossLocs = df_acrossLocs[new_order + [col for col in df_acrossLocs.columns if col not in new_order]]

        # Save to CSV
        df_acrossLocs.to_csv(f'df_{dataset_name}_batch{batch_counter}.csv', index=False)
 